In [ ]:
import os
import cv2
import dlib
import numpy as np
import argparse
from contextlib import contextmanager
from keras.utils.data_utils import get_file
import csv
import sys, tqdm
import pandas as pd
sys.path.append(r'./age-gender/')
from wide_resnet import WideResNet

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
import math
import re, tqdm
from collections import Counter

In [ ]:
pretrained_model = r"./weights.29-3.76_utk.hdf5"
modhash = 'fbe63257a054c1c5466cfd7bf14646d6'

images = os.listdir(r"./input_images/")
images = ["./input_images/"+name for name in images]

class MyImage:
    def __init__(self, img_name):
        self.img = cv2.imread(img_name)
        self.__name = img_name

    def __str__(self):
        return self.__name

In [ ]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)


def yield_images():
    for img in images:
        yield(MyImage(img))

In [ ]:
depth = 16
k = 8
weight_file = pretrained_model
margin = 0.4

# for face detection
detector = dlib.get_frontal_face_detector()

# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(pretrained_model)

In [ ]:
results_df = pd.DataFrame()
failed = []

for img in tqdm.tqdm(yield_images()):
    name = str(img)
    img = img.img
    
    try:
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    except:
        failed.append(img)
        continue
    
    img_h, img_w, _ = np.shape(input_img)
    
    # detect faces using dlib detector
    detected = detector(input_img, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(img[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))

        # predict ages and genders of the detected faces
        results = model.predict(faces)
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        gender = "F" if predicted_genders[0][0] > 0.5 else "M"

        results_df = results_df.append([[name.split('/')[-1], int(predicted_ages[0]), gender]])

In [ ]:
results_df.shape

In [ ]:
results_df.head()

In [ ]:
## Join your input data with the age/gender columns

In [ ]:
results_df.to_csv('./age_gender.csv', index=False)

In [ ]:
names = pd.read_csv('./Lista_de_nomes_2017_11.csv')
names.head()

In [ ]:
results=pd.read_csv('./age_gender.csv')
results['is_portuguese_name'] = 0
results['name'] = results['fname'] +' '+results['lname']
results.head()

In [ ]:
WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text.lower())
    return Counter(words)

for i in tqdm.tqdm(range(results.shape[0])):
    for j in range(names.shape[0]):
        try:
            if get_cosine(text_to_vector(results.loc[i,'name']), text_to_vector(names.loc[j, 'name'])) > 0.1:
                results.loc[i,'is_portuguese_name'] = 1
        except:
            pass

In [ ]:
results['location_type'] = None
portugal_list = ['abrantes', 'águeda', 'aguiar da beira', 'alandroal', 'albergaria-a-velha', 'albufeira', 'alcácer do sal', 'alcanena', 'alcobaça', 'alcochete', 'alcoutim', 'alenquer', 'alfândega da fé', 'alijó', 'aljezur', 'aljustrel', 'almada', 'almeida', 'almeirim', 'almodôvar', 'alpiarça', 'alter do chão', 'alvaiázere', 'alvito', 'amadora', 'amarante', 'amares', 'anadia', 'angra do heroísmo', 'ansião', 'arcos de valdevez', 'arganil', 'armamar', 'arouca', 'arraiolos', 'arronches', 'arruda dos vinhos', 'aveiro', 'avis', 'azambuja', 'baião', 'barcelos', 'barrancos', 'barreiro', 'batalha', 'beja', 'belmonte', 'benavente', 'bombarral', 'borba', 'boticas', 'braga', 'bragança', 'cabeceiras de basto', 'cadaval', 'caldas da rainha', 'calheta (madeira)', 'calheta (são jorge)', 'caminha', 'campo maior', 'cantanhede', 'carrazeda de ansiães', 'carregal do sal', 'cartaxo', 'cascais', 'castanheira de pêra', 'castelo branco', 'castelo de paiva', 'castelo de vide', 'castro daire', 'castro marim', 'castro verde', 'celorico da beira', 'celorico de basto', 'chamusca', 'chaves', 'cinfães', 'coimbra', 'condeixa-a-nova', 'constância', 'coruche', 'corvo', 'covilhã', 'crato', 'cuba', 'câmara de lobos', 'elvas', 'entroncamento', 'espinho', 'esposende', 'estarreja', 'estremoz', 'évora', 'fafe', 'faro', 'felgueiras', 'ferreira do alentejo', 'ferreira do zêzere', 'figueira da foz', 'figueira de castelo rodrigo', 'figueiró dos vinhos', 'fornos de algodres', 'freixo de espada à cinta', 'fronteira', 'funchal', 'fundão', 'gavião', 'golegã', 'gondomar', 'gouveia', 'grândola', 'guarda', 'guimarães', 'góis', 'horta', 'idanha-a-nova', 'ílhavo', 'lagoa (algarve)', 'lagoa (são miguel)', 'lagos', 'lajes das flores', 'lajes do pico', 'lamego', 'leiria', 'lisboa', 'loulé', 'loures', 'lourinhã', 'lousã', 'lousada', 'mação', 'macedo de cavaleiros', 'machico', 'madalena', 'mafra', 'maia', 'mangualde', 'manteigas', 'marco de canaveses', 'marinha grande', 'marvão', 'matosinhos', 'mealhada', 'meda', 'melgaço', 'mesão frio', 'mira', 'miranda do corvo', 'miranda do douro', 'mirandela', 'mogadouro', 'moimenta da beira', 'moita', 'monção', 'monchique', 'mondim de basto', 'monforte', 'montalegre', 'montemor-o-novo', 'montemor-o-velho', 'montijo', 'mora', 'mortágua', 'moura', 'mourão', 'murça', 'murtosa', 'mértola', 'nazaré', 'nelas', 'nisa', 'nordeste', 'óbidos', 'odemira', 'odivelas', 'oeiras', 'oleiros', 'olhão', 'oliveira de azeméis', 'oliveira de frades', 'oliveira do bairro', 'oliveira do hospital', 'ourique', 'ourém', 'ovar', 'paços de ferreira', 'palmela', 'pampilhosa da serra', 'paredes', 'paredes de coura', 'pedrógão grande', 'penacova', 'penafiel', 'penalva do castelo', 'penamacor', 'penedono', 'penela', 'peniche', 'peso da régua', 'pinhel', 'pombal', 'ponta delgada', 'ponta do sol', 'ponte da barca', 'ponte de lima', 'ponte de sor', 'portalegre', 'portel', 'portimão', 'porto', 'porto moniz', 'porto santo', 'porto de mós', 'povoação', 'praia da vitória', 'proença-a-nova', 'póvoa de lanhoso', 'póvoa de varzim', 'redondo', 'reguengos de monsaraz', 'resende', 'ribeira brava', 'ribeira grande', 'ribeira de pena', 'rio maior', 'sabrosa', 'sabugal', 'salvaterra de magos', 'santa comba dão', 'santa cruz', 'santa cruz da graciosa', 'santa cruz das flores', 'santa maria da feira', 'santa marta de penaguião', 'santana', 'santarém', 'santiago do cacém', 'santo tirso', 'são brás de alportel', 'são joão da madeira', 'são joão da pesqueira', 'são pedro do sul', 'são roque do pico', 'são vicente', 'sardoal', 'sátão', 'seia', 'seixal', 'sernancelhe', 'serpa', 'sertã', 'sesimbra', 'setúbal', 'sever do vouga', 'silves', 'sines', 'sintra', 'sobral de monte agraço', 'soure', 'sousel', 'tábua', 'tabuaço', 'tarouca', 'tavira', 'terras de bouro', 'tomar', 'tondela', 'torre de moncorvo', 'torres novas', 'torres vedras', 'trancoso', 'trofa', 'vagos', 'vale de cambra', 'valença', 'valongo', 'valpaços', 'velas', 'vendas novas', 'viana do alentejo', 'viana do castelo', 'vidigueira', 'vieira do minho', 'vila flor', 'vila franca de xira', 'vila franca do campo', 'vila nova da barquinha', 'vila nova de cerveira', 'vila nova de famalicão', 'vila nova de foz côa', 'vila nova de gaia', 'vila nova de paiva', 'vila nova de poiares', 'vila pouca de aguiar', 'vila real', 'vila real de santo antónio', 'vila velha de ródão', 'vila verde', 'vila viçosa', 'vila de rei', 'vila do bispo', 'vila do conde', 'vila do porto', 'vimioso', 'vinhais', 'viseu', 'vizela', 'vouzela']
lisbon_list = ['alcochete','almada','barreiro','amadora','cascais','lisboa','loures','mafra','moita','montijo','odivelas','oeiras','palmela','seixal','sintra','sesimbra','setúbal e vila franca de xira','lsibon']

In [ ]:
for i in tqdm.tqdm(range(results.shape[0])):
    try:
        location = WORD.findall(results.loc[i,'region'].lower())
        if 'lisboa' in location or 'lisbon' in location:
            results.loc[i,'location_type'] = 'lisbon'
        else:
            results.loc[i,'location_type'] = 'non_lisbon'
    except:
        results.loc[i,'location_type'] = 'non_lisbon'    

In [ ]:
results.head()

In [ ]:
results.to_csv('./socio_demographics.csv', index=False)